<a href="https://colab.research.google.com/github/skashyap777/Digifoxintern/blob/main/ReDEPLOY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving label_encoders.pkl to label_encoders.pkl
Saving feature_modes.pkl to feature_modes.pkl
Saving feature_names.txt to feature_names.txt
Saving rf_model.pkl to rf_model.pkl
Saving selected_features.pkl to selected_features.pkl


In [2]:
pip install --ignore-installed blinker

In [3]:
!pip install -q streamlit
!npm install -g localtunnel
!pip install -q joblib pandas numpy scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 4s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸

In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set page configuration
st.set_page_config(
    page_title="Gene Mutation Pathogenicity Predictor",
    page_icon="🧬",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for better styling
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #2E8B57;
        text-align: center;
        margin-bottom: 2rem;
    }
    .sub-header {
        font-size: 1.5rem;
        color: #4682B4;
        margin-bottom: 1rem;
    }
    .feature-section {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        margin-bottom: 1rem;
    }
    .prediction-result {
        font-size: 2rem;
        text-align: center;
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 1rem 0;
    }
    .pathogenic {
        background-color: #ffebee;
        color: #c62828;
        border: 2px solid #ef5350;
    }
    .benign {
        background-color: #e8f5e8;
        color: #2e7d32;
        border: 2px solid #66bb6a;
    }
    .upload-section {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 2rem;
        border-radius: 1rem;
        margin: 1rem 0;
        text-align: center;
    }
    .upload-section h3 {
        color: white;
        margin-bottom: 1rem;
    }
    .file-upload-info {
        background-color: #e3f2fd;
        padding: 1rem;
        border-radius: 0.5rem;
        border-left: 4px solid #2196f3;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

# Load model and preprocessing objects
@st.cache_resource
def load_model_artifacts():
    try:
        model = joblib.load('rf_model.pkl')
        with open('selected_features.pkl', 'rb') as f:
            selected_features = pickle.load(f)
        with open('label_encoders.pkl', 'rb') as f:
            label_encoders = pickle.load(f)
        with open('feature_modes.pkl', 'rb') as f:
            feature_modes = pickle.load(f)
        st.info("Model and artifacts loaded successfully.")
        return model, selected_features, label_encoders, feature_modes
    except FileNotFoundError as e:
        st.error(f"File not found: {e}. Please upload rf_model.pkl, selected_features.pkl, label_encoders.pkl, and feature_modes.pkl to the Colab /content/ directory.")
        return None, None, None, None
    except Exception as e:
        st.error(f"Error loading artifacts: {e}. Check library versions (pandas==2.2.2, scikit-learn==1.5.2, joblib==1.4.2).")
        return None, None, None, None

# Main application
def main():
    st.markdown('<h1 class="main-header">🧬 Gene Mutation Pathogenicity Predictor</h1>', unsafe_allow_html=True)

    # Load model artifacts
    model, selected_features, label_encoders, feature_modes = load_model_artifacts()

    if model is None:
        st.stop()

    st.markdown("### Predict whether a gene mutation is **Pathogenic** or **Benign**")

    # Add prominent CSV upload section at the top
    st.markdown("""
    <div class="upload-section">
        <h3>📁 Quick CSV Upload for Batch Predictions</h3>
        <p>Upload your gene mutation CSV file to get instant pathogenicity predictions for multiple mutations!</p>
    </div>
    """, unsafe_allow_html=True)

    # CSV Upload section
    with st.expander("🚀 **UPLOAD CSV FILE FOR BATCH PREDICTIONS**", expanded=True):
        csv_upload_section(model, selected_features, label_encoders, feature_modes)

    st.markdown("---")

    # Sidebar for navigation
    st.sidebar.title("Navigation")
    page = st.sidebar.selectbox("Choose a page:", ["Single Prediction", "Advanced Batch Upload", "Feature Information", "CSV Format Guide"])

    if page == "Single Prediction":
        single_prediction_page(model, selected_features, label_encoders, feature_modes)
    elif page == "Advanced Batch Upload":
        batch_prediction_page(model, selected_features, label_encoders, feature_modes)
    elif page == "CSV Format Guide":
        csv_format_guide_page(selected_features, label_encoders, feature_modes)
    else:
        feature_information_page(selected_features)

def csv_upload_section(model, selected_features, label_encoders, feature_modes):
    """Quick CSV upload section for the main page"""
    st.markdown("**Upload your CSV file with gene mutation data:**")

    # File upload
    uploaded_file = st.file_uploader(
        "Choose a CSV file",
        type="csv",
        help="Upload a CSV file containing gene mutation data. Ensure columns match the model features."
    )

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
            st.success(f"✅ File uploaded successfully! {df.shape[0]} mutations found in {df.shape[1]} columns")

            # Show file info
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Total Rows", df.shape[0])
            with col2:
                st.metric("Total Columns", df.shape[1])
            with col3:
                matching_features = len([col for col in df.columns if col in selected_features])
                st.metric("Matching Features", f"{matching_features}/{len(selected_features)}")

            # Show preview
            st.markdown("**Data Preview:**")
            st.dataframe(df.head(10), use_container_width=True)

            # Batch size selection
            batch_size = st.number_input("Batch Size for Processing", min_value=100, max_value=1000, value=500, step=100)

            # Prediction button
            col1, col2 = st.columns([2, 1])
            with col1:
                if st.button("🔮 **PREDICT PATHOGENICITY FOR ALL MUTATIONS**", type="primary", use_container_width=True):
                    with st.spinner("Making predictions..."):
                        predictions, probabilities = make_batch_predictions(
                            df, model, selected_features, label_encoders, feature_modes, batch_size
                        )
                        if predictions:
                            display_batch_results(df, predictions, probabilities)

            with col2:
                if st.button("📋 Show Required Columns", use_container_width=True):
                    show_required_columns(selected_features, df.columns)

        except Exception as e:
            st.error(f"❌ Error processing file: {e}")
            st.info("Ensure your CSV is properly formatted with correct column names and data types.")

def show_required_columns(selected_features, uploaded_columns):
    """Show required columns vs uploaded columns"""
    st.markdown("### Column Comparison")

    col1, col2 = st.columns(2)

    with col1:
        st.markdown("**Available in your CSV:**")
        available = [col for col in uploaded_columns if col in selected_features]
        for col in available[:20]:
            st.write(f"✅ {col}")
        if len(available) > 20:
            st.write(f"... and {len(available) - 20} more")

    with col2:
        st.markdown("**Missing (will use defaults):**")
        missing = [col for col in selected_features if col not in uploaded_columns]
        for col in missing[:20]:
            st.write(f"⚠️ {col}")
        if len(missing) > 20:
            st.write(f"... and {len(missing) - 20} more")

def display_batch_results(original_df, predictions, probabilities):
    """Display batch prediction results"""
    results_df = original_df.copy()
    results_df['Predicted_Pathogenicity'] = predictions
    results_df['Pathogenic_Probability'] = [f"{p:.3f}" for p in probabilities]
    results_df['Confidence_Level'] = ['High' if p > 0.8 or p < 0.2 else 'Medium' if p > 0.6 or p < 0.4 else 'Low'
                                     for p in probabilities]

    st.markdown("## 🎯 Prediction Results")

    pathogenic_count = sum(1 for p in predictions if p == 'Pathogenic')
    benign_count = len(predictions) - pathogenic_count

    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric("Total Predictions", len(predictions))
    with col2:
        st.metric("🔴 Pathogenic", pathogenic_count, f"{pathogenic_count/len(predictions)*100:.1f}%")
    with col3:
        st.metric("🟢 Benign", benign_count, f"{benign_count/len(predictions)*100:.1f}%")
    with col4:
        high_conf = sum(1 for p in probabilities if p > 0.8 or p < 0.2)
        st.metric("High Confidence", high_conf, f"{high_conf/len(predictions)*100:.1f}%")

    st.markdown("### Filter Results:")
    col1, col2 = st.columns(2)
    with col1:
        filter_prediction = st.selectbox("Filter by Prediction:", ["All", "Pathogenic", "Benign"])
    with col2:
        filter_confidence = st.selectbox("Filter by Confidence:", ["All", "High", "Medium", "Low"])

    filtered_df = results_df.copy()
    if filter_prediction != "All":
        filtered_df = filtered_df[filtered_df['Predicted_Pathogenicity'] == filter_prediction]
    if filter_confidence != "All":
        filtered_df = filtered_df[filtered_df['Confidence_Level'] == filter_confidence]

    st.markdown(f"**Showing {len(filtered_df)} of {len(results_df)} predictions:**")
    st.dataframe(filtered_df, use_container_width=True)

    st.markdown("### 📥 Download Results:")
    col1, col2 = st.columns(2)

    with col1:
        csv_data = results_df.to_csv(index=False)
        st.download_button(
            label="📄 Download All Results (CSV)",
            data=csv_data,
            file_name=f"mutation_predictions_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
            mime="text/csv",
            use_container_width=True
        )

    with col2:
        pathogenic_only = results_df[results_df['Predicted_Pathogenicity'] == 'Pathogenic']
        if len(pathogenic_only) > 0:
            pathogenic_csv = pathogenic_only.to_csv(index=False)
            st.download_button(
                label="🔴 Download Pathogenic Only (CSV)",
                data=pathogenic_csv,
                file_name=f"pathogenic_mutations_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv",
                use_container_width=True
            )

def csv_format_guide_page(selected_features, label_encoders, feature_modes):
    """Page explaining CSV format requirements"""
    st.markdown('<h2 class="sub-header">📋 CSV Format Guide</h2>', unsafe_allow_html=True)

    st.markdown("""
    <div class="file-upload-info">
        <h4>📁 How to format your CSV file for gene mutation prediction:</h4>
        <p>Your CSV file should contain gene mutation data with columns matching the model's expected features.
        Missing columns will use default values.</p>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("### 🎯 **Quick Start: Download Sample CSV Template**")

    sample_data = create_sample_csv_data(selected_features, label_encoders, feature_modes)
    sample_csv = pd.DataFrame(sample_data).to_csv(index=False)

    col1, col2 = st.columns(2)
    with col1:
        st.download_button(
            label="📥 Download Sample CSV Template",
            data=sample_csv,
            file_name="gene_mutation_template.csv",
            mime="text/csv",
            help="Download this template and fill it with your mutation data"
        )

    with col2:
        if st.button("👀 Preview Sample Data"):
            st.dataframe(pd.DataFrame(sample_data), use_container_width=True)

    st.markdown("### 📊 **Model Features by Category**")

    categories = {
        "🧬 Basic Mutation Info": [f for f in selected_features if any(x in f.lower() for x in ['mutation', 'allele', 'type', 'cpg'])],
        "📍 Genomic Position": [f for f in selected_features if any(x in f.lower() for x in ['hg', 'start', 'end', 'cdna', 'exon'])],
        "🔬 Protein Effects": [f for f in selected_features if 'protein' in f.lower()],
        "📊 Prediction Scores": [f for f in selected_features if any(x in f.lower() for x in ['sift', 'polyphen', 'provean', 'condel', 'mutassessor'])],
        "🏥 Clinical Data": [f for f in selected_features if any(x in f.lower() for x in ['disease', 'tumor', 'cancer', 'pathology', 'smoking'])],
    }

    for category, features in categories.items():
        if features:
            with st.expander(f"{category} ({len(features)} features)"):
                for i, feature in enumerate(sorted(features), 1):
                    st.write(f"{i}. `{feature}`")

    st.markdown("### 💡 **Tips for Best Results**")
    st.markdown("""
    - **Include as many relevant columns as possible** for better predictions
    - **Use consistent column names** matching the features above
    - **Handle missing values**: Use 'Unknown' or leave blank for categorical data
    - **Ensure numeric columns** contain numbers, not text
    - **Avoid special characters** in categorical values
    """)

def create_sample_csv_data(selected_features, label_encoders, feature_modes):
    """Create sample CSV data for download"""
    num_samples = 3
    sample_data = {}
    log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

    for feature in selected_features:
        if feature in label_encoders:
            classes = list(label_encoders[feature].classes_)
            sample_data[feature] = np.random.choice(classes, size=num_samples)
        else:
            mode_value = feature_modes.get(feature, 0)
            if isinstance(mode_value, (int, float)):
                values = np.random.uniform(low=max(0, mode_value - 1), high=mode_value + 1, size=num_samples)
                if feature in log_transformed_columns:
                    values = np.log1p(values)
                sample_data[feature] = values
            else:
                sample_data[feature] = [mode_value] * num_samples

    return sample_data

def single_prediction_page(model, selected_features, label_encoders, feature_modes):
    st.markdown('<h2 class="sub-header">Single Mutation Prediction</h2>', unsafe_allow_html=True)

    tab1, tab2, tab3, tab4, tab5 = st.tabs(["🧬 Basic Info", "📍 Genomic Position", "🔬 Protein Features", "📊 Scores & Predictions", "🏥 Clinical Data"])

    user_input = {}

    with tab1:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Basic Mutation Information**")

        col1, col2 = st.columns(2)

        with col1:
            if 'Mutation_Type_y' in selected_features and 'Mutation_Type_y' in label_encoders:
                user_input['Mutation_Type_y'] = st.selectbox("Mutation Type", list(label_encoders['Mutation_Type_y'].classes_))
            if 'Mutant_Allele_x' in selected_features and 'Mutant_Allele_x' in label_encoders:
                user_input['Mutant_Allele_x'] = st.selectbox("Mutant Allele", list(label_encoders['Mutant_Allele_x'].classes_))
            if 'Type' in selected_features and 'Type' in label_encoders:
                user_input['Type'] = st.selectbox("Variant Type", list(label_encoders['Type'].classes_))

        with col2:
            if 'CpG_y' in selected_features and 'CpG_y' in label_encoders:
                user_input['CpG_y'] = st.selectbox("CpG Site", list(label_encoders['CpG_y'].classes_))
            if 'Complexity' in selected_features and 'Complexity' in label_encoders:
                user_input['Complexity'] = st.selectbox("Mutation Complexity", list(label_encoders['Complexity'].classes_))
            if 'Py_Py_Doublets' in selected_features:
                user_input['Py_Py_Doublets'] = st.number_input("Py-Py Doublets", min_value=0, max_value=10, value=0, step=1)

        st.markdown('</div>', unsafe_allow_html=True)

    with tab2:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Genomic Position & Coordinates**")

        col1, col2, col3 = st.columns(3)

        with col1:
            if 'HG19_Start_x' in selected_features:
                user_input['HG19_Start_x'] = st.number_input("HG19 Start Position", min_value=1, value=17000000, step=1)
            if 'HG19_End_x' in selected_features:
                user_input['HG19_End_x'] = st.number_input("HG19 End Position", min_value=1, value=17000001, step=1)

        with col2:
            if 'HG38_Start' in selected_features:
                user_input['HG38_Start'] = st.number_input("HG38 Start Position", min_value=1, value=7500000, step=1)
            if 'HG38_End' in selected_features:
                user_input['HG38_End'] = st.number_input("HG38 End Position", min_value=1, value=7500001, step=1)

        with col3:
            if 'Start_cDNA_x' in selected_features:
                user_input['Start_cDNA_x'] = st.number_input("cDNA Start", min_value=1, value=100, step=1)
            if 'End_cDNA_x' in selected_features:
                user_input['End_cDNA_x'] = st.number_input("cDNA End", min_value=1, value=101, step=1)

        st.markdown('</div>', unsafe_allow_html=True)

    with tab3:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Protein Isoform Effects**")

        protein_features = [f for f in selected_features if f.startswith('Protein')]

        if protein_features:
            st.markdown("**TP53 Protein Isoform Predictions:**")
            cols = st.columns(3)

            for i, feature in enumerate(protein_features[:15]):
                with cols[i % 3]:
                    if feature in label_encoders:
                        user_input[feature] = st.selectbox(
                            feature.replace('Protein ', '').replace('_', ' '),
                            list(label_encoders[feature].classes_),
                            key=f"protein_{i}"
                        )
                    else:
                        user_input[feature] = st.selectbox(
                            feature.replace('Protein ', '').replace('_', ' '),
                            ['Neutral', 'Deleterious', 'Unknown'],
                            key=f"protein_{i}"
                        )

        st.markdown('</div>', unsafe_allow_html=True)

    with tab4:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Prediction Scores & Tools**")

        col1, col2 = st.columns(2)

        with col1:
            if 'Sift Score' in selected_features:
                user_input['Sift Score'] = st.slider("SIFT Score", 0.0, 1.0, 0.5, step=0.01)
            if 'Sift Prediction' in selected_features and 'Sift Prediction' in label_encoders:
                user_input['Sift Prediction'] = st.selectbox("SIFT Prediction", list(label_encoders['Sift Prediction'].classes_))
            if 'Polyphen' in selected_features and 'Polyphen' in label_encoders:
                user_input['Polyphen'] = st.selectbox("PolyPhen Prediction", list(label_encoders['Polyphen'].classes_))
            if 'Provean_Score_y' in selected_features:
                user_input['Provean_Score_y'] = st.slider("PROVEAN Score", -10.0, 10.0, -2.5, step=0.1)

        with col2:
            if 'Mutassessor_score_y' in selected_features:
                user_input['Mutassessor_score_y'] = st.slider("MutationAssessor Score", 0.0, 5.0, 2.5, step=0.1)
            if 'Condel_y' in selected_features:
                user_input['Condel_y'] = st.slider("Condel Score", 0.0, 1.0, 0.5, step=0.01)
            if 'MutPred_Splice_General_Score_y' in selected_features:
                user_input['MutPred_Splice_General_Score_y'] = st.slider("MutPred Splice Score", 0.0, 1.0, 0.5, step=0.01)

        st.markdown('</div>', unsafe_allow_html=True)

    with tab5:
        st.markdown('<div class="feature-section">', unsafe_allow_html=True)
        st.markdown("**Clinical & Environmental Data**")

        col1, col2 = st.columns(2)

        with col1:
            if 'Disease' in selected_features and 'Disease' in label_encoders:
                user_input['Disease'] = st.selectbox("Associated Disease", list(label_encoders['Disease'].classes_))
            if 'Sample_pathology' in selected_features and 'Sample_pathology' in label_encoders:
                user_input['Sample_pathology'] = st.selectbox("Sample Pathology", list(label_encoders['Sample_pathology'].classes_))
            if 'Smoking' in selected_features and 'Smoking' in label_encoders:
                user_input['Smoking'] = st.selectbox("Smoking History", list(label_encoders['Smoking'].classes_))

        with col2:
            if 'Tumor_Stat' in selected_features and 'Tumor_Stat' in label_encoders:
                user_input['Tumor_Stat'] = st.selectbox("Tumor Status", list(label_encoders['Tumor_Stat'].classes_))
            if 'Germline_Stat' in selected_features and 'Germline_Stat' in label_encoders:
                user_input['Germline_Stat'] = st.selectbox("Germline Status", list(label_encoders['Germline_Stat'].classes_))
            if 'Exposure' in selected_features and 'Exposure' in label_encoders:
                user_input['Exposure'] = st.selectbox("Environmental Exposure", list(label_encoders['Exposure'].classes_))

        st.markdown('</div>', unsafe_allow_html=True)

    for feature in selected_features:
        if feature not in user_input:
            if feature in feature_modes:
                user_input[feature] = feature_modes[feature]
            else:
                user_input[feature] = 0.0

    st.markdown("---")
    if st.button("🔮 Predict Pathogenicity", type="primary", use_container_width=True):
        prediction, probability = make_prediction(user_input, model, selected_features, label_encoders, feature_modes)
        display_prediction_result(prediction, probability)

def batch_prediction_page(model, selected_features, label_encoders, feature_modes):
    st.markdown('<h2 class="sub-header">Advanced Batch Mutation Prediction</h2>', unsafe_allow_html=True)

    st.markdown("Upload a CSV file with mutation data for batch prediction with advanced options.")

    uploaded_file = st.file_uploader("Choose a CSV file", type="csv")

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
            st.success(f"File uploaded successfully! Shape: {df.shape}")

            st.markdown("**Preview of uploaded data:**")
            st.dataframe(df.head())

            batch_size = st.slider("Batch Size for Processing", min_value=100, max_value=1000, value=500, step=100)

            if st.button("🚀 Run Batch Prediction", type="primary"):
                predictions, probabilities = make_batch_predictions(
                    df, model, selected_features, label_encoders, feature_modes, batch_size
                )

                results_df = df.copy()
                results_df['Predicted_Pathogenicity'] = predictions
                results_df['Pathogenic_Probability'] = [f"{p:.4f}" for p in probabilities]

                st.markdown("**Prediction Results:**")
                st.dataframe(results_df, use_container_width=True)

                csv = results_df.to_csv(index=False)
                st.download_button(
                    label="📥 Download Results as CSV",
                    data=csv,
                    file_name="mutation_predictions.csv",
                    mime="text/csv",
                    use_container_width=True
                )

                st.markdown("**Prediction Summary:**")
                col1, col2, col3 = st.columns(3)
                with col1:
                    st.metric("Total Predictions", len(predictions))
                with col2:
                    st.metric("Pathogenic", sum(1 for p in predictions if p == 'Pathogenic'))
                with col3:
                    st.metric("Benign", sum(1 for p in predictions if p == 'Benign'))

        except Exception as e:
            st.error(f"Error processing file: {e}")
            st.info("Please check that your CSV file contains valid data and correct column names.")

def feature_information_page(selected_features):
    st.markdown('<h2 class="sub-header">Feature Information</h2>', unsafe_allow_html=True)

    st.markdown("This page provides information about the 100 features used in the model.")

    categories = {
        "Genomic Position": [f for f in selected_features if any(x in f.lower() for x in ['hg', 'start', 'end', 'cdna', 'exon'])],
        "Protein Effects": [f for f in selected_features if 'protein' in f.lower()],
        "Prediction Scores": [f for f in selected_features if any(x in f.lower() for x in ['sift', 'polyphen', 'provean', 'condel', 'mutassessor'])],
        "Mutation Details": [f for f in selected_features if any(x in f.lower() for x in ['mutation', 'allele', 'codon', 'variant'])],
        "Clinical Data": [f for f in selected_features if any(x in f.lower() for x in ['disease', 'tumor', 'cancer', 'pathology', 'smoking'])],
        "Other Features": []
    }

    categorized = set()
    for cat_features in categories.values():
        categorized.update(cat_features)

    categories["Other Features"] = [f for f in selected_features if f not in categorized]

    for category, features in categories.items():
        if features:
            with st.expander(f"{category} ({len(features)} features)"):
                for feature in sorted(features):
                    st.write(f"• {feature}")

def preprocess_input(user_input, selected_features, label_encoders, feature_modes):
    """Preprocess user input to match model expectations"""
    processed_input = {}
    log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

    for feature in selected_features:
        if feature in user_input:
            value = user_input[feature]

            if feature in label_encoders:
                try:
                    if isinstance(value, str):
                        processed_input[feature] = label_encoders[feature].transform([value])[0]
                    else:
                        processed_input[feature] = value
                except Exception:
                    processed_input[feature] = label_encoders[feature].transform([label_encoders[feature].classes_[0]])[0]
                    st.warning(f"Invalid value for {feature}: {value}. Using default encoded value.")
            else:
                if feature in log_transformed_columns and isinstance(value, (int, float)):
                    processed_input[feature] = np.log1p(value)
                else:
                    processed_input[feature] = value
        else:
            default_value = feature_modes.get(feature, 0.0)
            if feature in log_transformed_columns and isinstance(default_value, (int, float)):
                processed_input[feature] = np.log1p(default_value)
            else:
                processed_input[feature] = default_value

    return processed_input

def make_prediction(user_input, model, selected_features, label_encoders, feature_modes):
    """Make a single prediction"""
    try:
        processed_input = preprocess_input(user_input, selected_features, label_encoders, feature_modes)

        input_df = pd.DataFrame([processed_input])
        input_df = input_df[selected_features]

        prediction_proba = model.predict_proba(input_df)[0]
        prediction = model.predict(input_df)[0]

        pred_label = 'Pathogenic' if prediction == 1 else 'Benign'
        confidence = float(max(prediction_proba))

        return pred_label, confidence

    except Exception as e:
        st.error(f"Error making prediction: {str(e)}")
        st.error("Please check input data format and try again.")
        return None, 0.0

def make_batch_predictions(df, model, selected_features, label_encoders, feature_modes, batch_size=500):
    """Make batch predictions"""
    errors = []
    try:
        input_df = df.copy()
        log_transformed_columns = ['Leukemia_Lymhoma_Freq', 'Solid_Tumor_Freq', 'Tumor_Freq', 'Cell_line_Freq']

        for feature in selected_features:
            if feature not in input_df.columns:
                input_df[feature] = feature_modes.get(feature, 0)

        for feature in selected_features:
            if feature in input_df.columns:
                if feature in label_encoders:
                    try:
                        input_df[feature] = input_df[feature].fillna('Unknown').astype(str)
                        encoder_classes = [str(c) for c in label_encoders[feature].classes_]
                        def safe_transform(value):
                            value = str(value).strip()
                            return label_encoders[feature].transform([value])[0] if value in encoder_classes else label_encoders[feature].transform([encoder_classes[0]])[0]
                        input_df[feature] = input_df[feature].apply(safe_transform)
                    except Exception as enc_error:
                        errors.append(f"Error encoding feature {feature}: {str(enc_error)}")
                        input_df[feature] = feature_modes.get(feature, 0)
                elif feature in log_transformed_columns:
                    input_df[feature] = pd.to_numeric(input_df[feature], errors='coerce').abs().fillna(0)
                    input_df[feature] = np.log1p(input_df[feature])

        all_predictions = []
        all_probabilities = []
        progress_bar = st.progress(0)
        total_batches = (len(input_df) + batch_size - 1) // batch_size

        for i in range(0, len(input_df), batch_size):
            batch_df = input_df.iloc[i:i+batch_size][selected_features]
            batch_df = batch_df.fillna(0)
            predictions = model.predict(batch_df)
            probabilities = model.predict_proba(batch_df)
            all_predictions.extend(['Pathogenic' if p == 1 else 'Benign' for p in predictions])
            all_probabilities.extend(probabilities[:, 1] if probabilities.shape[1] >= 2 else probabilities[:, 0])
            progress_bar.progress(min((i + batch_size) / len(input_df), 1.0))

        progress_bar.empty()

        if errors:
            with open('prediction_errors.txt', 'w') as f:
                f.write("\n".join(errors))
            st.markdown("### Download Error Log")
            st.download_button(
                label="📋 Download Error Log",
                data=open('prediction_errors.txt', 'rb'),
                file_name="prediction_errors.txt",
                mime="text/plain",
                use_container_width=True
            )

        return all_predictions, all_probabilities

    except Exception as e:
        errors.append(str(e))
        with open('prediction_errors.txt', 'w') as f:
            f.write("\n".join(errors))
        st.error(f"Error in batch predictions: {str(e)}")
        st.error("Check your CSV for valid data (correct columns, data types).")
        st.download_button(
            label="📋 Download Error Log",
            data=open('prediction_errors.txt', 'rb'),
            file_name="prediction_errors.txt",
            mime="text/plain"
        )
        return [], []

def display_prediction_result(prediction, probability):
    """Display prediction result with styling"""
    if prediction is None:
        st.error("Prediction failed. Please check your input data.")
        return

    if prediction == "Pathogenic":
        st.markdown(f'''
        <div class="prediction-result pathogenic">
            <strong>⚠️ PATHOGENIC</strong><br>
            Confidence: {probability:.2%}
        </div>
        ''', unsafe_allow_html=True)

        st.warning("⚠️ This mutation is predicted to be **PATHOGENIC**. Consider further clinical evaluation.")

    elif prediction == "Benign":
        st.markdown(f'''
        <div class="prediction-result benign">
            <strong>✔️ BENIGN</strong><br>
            Confidence: {probability:.2%}
        </div>
        ''', unsafe_allow_html=True)

        st.success("✔️ This mutation is predicted to be **BENIGN**. Low likelihood of pathogenicity.")

if __name__ == "__main__":
    main()


Writing app.py


In [5]:
!wget -q -O - ipv4.icanhazip.com
!streamlit run app.py & sleep 5 && npx localtunnel --port 8501

34.9.113.65



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.9.113.65:8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://lovely-pots-matter.loca.lt
  Stopping...
^C
